In [86]:
import pandas as pd

In [87]:
betfair_results=pd.read_csv("Betting/BetHistory.csv")
betfair_results

,Market,Selection,Bid type,Bet ID,Bet placed,Unnamed: 5,Persistence,Odds req.,Stake (AUD),Liability (AUD),Avg. odds matched,Profit/Loss (AUD)
0,AU / Caulfield (AUS) 26th Feb / 17:05 To Be Pl...,5. Marabi,Stake_lay,2.600000e+11,26/02/2022 17:03,NaN,C,1.50,583,291.5,1.50,-291.5
1,AU / Caulfield (AUS) 26th Feb / 17:05 R8 1100m...,5. Marabi,Stake_lay,2.600000e+11,26/02/2022 17:02,NaN,C,2.80,"1,166.00","2,098.80",2.80,"-2,098.80"
2,AU / Randwick (AUS) 26th Feb / 16:05 To Be Placed,1. Think It Over,Full_lay,2.600000e+11,26/02/2022 16:04,NaN,C,1.82,436.05,357.56,1.82,436.05
3,AU / Randwick (AUS) 26th Feb / 16:05 R7 1600m ...,1. Think It Over,Full_lay,2.600000e+11,26/02/2022 16:04,NaN,C,5.70,246.86,"1,160.24",5.70,246.86
4,AU / Caulfield (AUS) 26th Feb / 15:45 To Be Pl...,4. Tofane,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,1.28,146.62,41.05,1.28,-41.06
5,AU / Caulfield (AUS) 26th Feb / 15:45 To Be Pl...,5. Sierra Sue,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,3.60,943.4,"2,452.84",3.60,"-2,452.83"
6,AU / Caulfield (AUS) 26th Feb / 15:45 R6 1400m...,4. Tofane,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,2.38,858.6,"1,184.87",2.38,858.6
7,AU / Caulfield (AUS) 26th Feb / 15:45 R6 1400m...,5. Sierra Sue,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,16.50,371,"5,750.50",16.50,"-5,750.53"
8,AU / Randwick (AUS) 26th Feb / 14:45 To Be Placed,2. Ellsberg,Stake_lay,2.600000e+11,26/02/2022 14:43,NaN,C,1.28,150.91,42.25,1.28,-42.29
9,AU / Randwick (AUS) 26th Feb / 14:45 R5 1300m ...,2. Ellsberg,Stake_lay,2.600000e+11,26/02/2022 14:43,NaN,C,2.76,423.64,745.61,2.76,-745.59


In [88]:
commission_dict={"Randwick":0.1,"Caulfield":0.06}

# Clean Betfair Spreadsheet

In [89]:
betfair_results["Selection"]=betfair_results["Selection"].str.translate(str.maketrans('', '', "0123456789.")).str.strip()


In [90]:
def has_place(MarketStr):
    """If market string has a place - marks as a place bet. Otherwise win"""
    if "Placed" in MarketStr:
        return "Place"
    return "Win"

betfair_results["Market Type"]=betfair_results["Market"].apply(has_place)

def convert_to_float(ProfitStr):
    if ProfitStr[0]=="(":
        return -float(ProfitStr[1:-1].replace(",",""))
    return float(ProfitStr.replace(",",""))



betfair_results["Profit/Loss (AUD)"] = betfair_results["Profit/Loss (AUD)"].apply(convert_to_float)
betfair_results["Liability (AUD)"] = betfair_results["Liability (AUD)"].apply(convert_to_float)
betfair_results["Stake (AUD)"] =  betfair_results["Stake (AUD)"].apply(convert_to_float)
betfair_results["Venue"]=betfair_results["Market"].apply(lambda x:x.split()[2])

In [91]:
# Calculating weighted average odds (won't be correct until the next groupby)

temp_groupby=betfair_results.loc[:,["Selection","Market Type","Liability (AUD)"]].groupby(["Selection","Market Type"]).sum()
temp_groupby=temp_groupby.rename({"Liability (AUD)":"Total"},axis=1)
betfair_results=betfair_results.join(temp_groupby,on=["Selection","Market Type"])
betfair_results["Weighted Avg. odds matched"]=(betfair_results["Liability (AUD)"]/betfair_results["Total"]) *betfair_results["Avg. odds matched"]


In [92]:
#Commission calc
market_resultDf=betfair_results.loc[:,[ 'Market','Profit/Loss (AUD)']].groupby(by=["Market"]).sum()
market_resultDf=market_resultDf.rename({'Profit/Loss (AUD)':"Market Result"},axis=1)
betfair_results=betfair_results.join(market_resultDf,on='Market')


betfair_results_no_neg=betfair_results[betfair_results["Profit/Loss (AUD)"]>0].loc[:,["Market","Profit/Loss (AUD)"]].groupby("Market").sum().rename({'Profit/Loss (AUD)':"Market Result No neg"},axis=1)
betfair_results=betfair_results.join(betfair_results_no_neg,on='Market')
betfair_results

,Market,Selection,Bid type,Bet ID,Bet placed,Unnamed: 5,Persistence,Odds req.,Stake (AUD),Liability (AUD),Avg. odds matched,Profit/Loss (AUD),Market Type,Venue,Total,Weighted Avg. odds matched,Market Result,Market Result No neg
0,AU / Caulfield (AUS) 26th Feb / 17:05 To Be Pl...,Marabi,Stake_lay,2.600000e+11,26/02/2022 17:03,NaN,C,1.50,583.00,291.50,1.50,-291.50,Place,Caulfield,291.50,1.500000,-291.50,NaN
1,AU / Caulfield (AUS) 26th Feb / 17:05 R8 1100m...,Marabi,Stake_lay,2.600000e+11,26/02/2022 17:02,NaN,C,2.80,1166.00,2098.80,2.80,-2098.80,Win,Caulfield,2098.80,2.800000,-2098.80,NaN
2,AU / Randwick (AUS) 26th Feb / 16:05 To Be Placed,Think It Over,Full_lay,2.600000e+11,26/02/2022 16:04,NaN,C,1.82,436.05,357.56,1.82,436.05,Place,Randwick,357.56,1.820000,436.05,436.05
3,AU / Randwick (AUS) 26th Feb / 16:05 R7 1600m ...,Think It Over,Full_lay,2.600000e+11,26/02/2022 16:04,NaN,C,5.70,246.86,1160.24,5.70,246.86,Win,Randwick,1160.24,5.700000,246.86,246.86
4,AU / Caulfield (AUS) 26th Feb / 15:45 To Be Pl...,Tofane,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,1.28,146.62,41.05,1.28,-41.06,Place,Caulfield,41.05,1.280000,-2493.89,NaN
5,AU / Caulfield (AUS) 26th Feb / 15:45 To Be Pl...,Sierra Sue,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,3.60,943.40,2452.84,3.60,-2452.83,Place,Caulfield,2452.84,3.600000,-2493.89,NaN
6,AU / Caulfield (AUS) 26th Feb / 15:45 R6 1400m...,Tofane,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,2.38,858.60,1184.87,2.38,858.60,Win,Caulfield,1184.87,2.380000,-4891.93,858.60
7,AU / Caulfield (AUS) 26th Feb / 15:45 R6 1400m...,Sierra Sue,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,16.50,371.00,5750.50,16.50,-5750.53,Win,Caulfield,5750.50,16.500000,-4891.93,858.60
8,AU / Randwick (AUS) 26th Feb / 14:45 To Be Placed,Ellsberg,Stake_lay,2.600000e+11,26/02/2022 14:43,NaN,C,1.28,150.91,42.25,1.28,-42.29,Place,Randwick,42.25,1.280000,-42.29,NaN
9,AU / Randwick (AUS) 26th Feb / 14:45 R5 1300m ...,Ellsberg,Stake_lay,2.600000e+11,26/02/2022 14:43,NaN,C,2.76,423.64,745.61,2.76,-745.59,Win,Randwick,904.01,2.276395,-903.99,NaN


In [93]:
def calc_commission(df):
    if df["Market Result"]<=0:
        return 0
    if df["Profit/Loss (AUD)"]<=0:
        return 0
    commission=df["Market Result"]*commission_dict[df["Venue"]]
    return commission * (df["Profit/Loss (AUD)"]/df["Market Result No neg"])

betfair_results["Commission Paid"]=betfair_results.apply(calc_commission,axis=1)

In [94]:
betfair_results

,Market,Selection,Bid type,Bet ID,Bet placed,Unnamed: 5,Persistence,Odds req.,Stake (AUD),Liability (AUD),Avg. odds matched,Profit/Loss (AUD),Market Type,Venue,Total,Weighted Avg. odds matched,Market Result,Market Result No neg,Commission Paid
0,AU / Caulfield (AUS) 26th Feb / 17:05 To Be Pl...,Marabi,Stake_lay,2.600000e+11,26/02/2022 17:03,NaN,C,1.50,583.00,291.50,1.50,-291.50,Place,Caulfield,291.50,1.500000,-291.50,NaN,0.000
1,AU / Caulfield (AUS) 26th Feb / 17:05 R8 1100m...,Marabi,Stake_lay,2.600000e+11,26/02/2022 17:02,NaN,C,2.80,1166.00,2098.80,2.80,-2098.80,Win,Caulfield,2098.80,2.800000,-2098.80,NaN,0.000
2,AU / Randwick (AUS) 26th Feb / 16:05 To Be Placed,Think It Over,Full_lay,2.600000e+11,26/02/2022 16:04,NaN,C,1.82,436.05,357.56,1.82,436.05,Place,Randwick,357.56,1.820000,436.05,436.05,43.605
3,AU / Randwick (AUS) 26th Feb / 16:05 R7 1600m ...,Think It Over,Full_lay,2.600000e+11,26/02/2022 16:04,NaN,C,5.70,246.86,1160.24,5.70,246.86,Win,Randwick,1160.24,5.700000,246.86,246.86,24.686
4,AU / Caulfield (AUS) 26th Feb / 15:45 To Be Pl...,Tofane,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,1.28,146.62,41.05,1.28,-41.06,Place,Caulfield,41.05,1.280000,-2493.89,NaN,0.000
5,AU / Caulfield (AUS) 26th Feb / 15:45 To Be Pl...,Sierra Sue,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,3.60,943.40,2452.84,3.60,-2452.83,Place,Caulfield,2452.84,3.600000,-2493.89,NaN,0.000
6,AU / Caulfield (AUS) 26th Feb / 15:45 R6 1400m...,Tofane,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,2.38,858.60,1184.87,2.38,858.60,Win,Caulfield,1184.87,2.380000,-4891.93,858.60,0.000
7,AU / Caulfield (AUS) 26th Feb / 15:45 R6 1400m...,Sierra Sue,Stake_lay,2.600000e+11,26/02/2022 15:43,NaN,C,16.50,371.00,5750.50,16.50,-5750.53,Win,Caulfield,5750.50,16.500000,-4891.93,858.60,0.000
8,AU / Randwick (AUS) 26th Feb / 14:45 To Be Placed,Ellsberg,Stake_lay,2.600000e+11,26/02/2022 14:43,NaN,C,1.28,150.91,42.25,1.28,-42.29,Place,Randwick,42.25,1.280000,-42.29,NaN,0.000
9,AU / Randwick (AUS) 26th Feb / 14:45 R5 1300m ...,Ellsberg,Stake_lay,2.600000e+11,26/02/2022 14:43,NaN,C,2.76,423.64,745.61,2.76,-745.59,Win,Randwick,904.01,2.276395,-903.99,NaN,0.000


In [95]:
betfair_results=betfair_results.loc[:,[ 'Selection','Market Type','Bid type', 'Profit/Loss (AUD)', "Avg. odds matched","Commission Paid","Venue","Stake (AUD)"]]
betfair_results=betfair_results.groupby(by=["Selection","Market Type",'Bid type',"Venue"],as_index=False).aggregate({'Profit/Loss (AUD)':'sum',"Commission Paid":'sum',"Avg. odds matched":'mean',"Stake (AUD)":"sum"})
betfair_results["Profit ex. Commission"]=betfair_results['Profit/Loss (AUD)']-betfair_results["Commission Paid"]
betfair_results

,Selection,Market Type,Bid type,Venue,Profit/Loss (AUD),Commission Paid,Avg. odds matched,Stake (AUD),Profit ex. Commission
0,Ellsberg,Place,Stake_lay,Randwick,-42.29,0.000,1.28,150.91,-42.290
1,Ellsberg,Win,Stake_lay,Randwick,-903.99,0.000,2.76,513.64,-903.990
2,Marabi,Place,Stake_lay,Caulfield,-291.50,0.000,1.50,583.00,-291.500
3,Marabi,Win,Stake_lay,Caulfield,-2098.80,0.000,2.80,1166.00,-2098.800
4,Sierra Sue,Place,Stake_lay,Caulfield,-2452.83,0.000,3.60,943.40,-2452.830
5,Sierra Sue,Win,Stake_lay,Caulfield,-5750.53,0.000,16.50,371.00,-5750.530
6,Think It Over,Place,Full_lay,Randwick,436.05,43.605,1.82,436.05,392.445
7,Think It Over,Win,Full_lay,Randwick,246.86,24.686,5.70,246.86,222.174
8,Tofane,Place,Stake_lay,Caulfield,-41.06,0.000,1.28,146.62,-41.060
9,Tofane,Win,Stake_lay,Caulfield,858.60,0.000,2.38,858.60,858.600


In [96]:
betting_results=pd.read_excel("Betting/Futures-Stakes.xlsx",sheet_name="Reconcilliation")
betting_results=betting_results.loc[:,["Horse","Odds","Victor Stake","Result","Notes"]]
betting_results.head()


,Horse,Odds,Victor Stake,Result,Notes
0,Bella Nipotina (Place),6.00,133.0,Lost,Eligibile for 5% profit rake
1,Collette,7.00,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake"
2,Collette (Place),2.50,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake"
3,Counttheheadlights,41.00,34.5,Lost,NaN
4,Ellsberg,6.48,300.0,Win,"Placed by Victor, Eligibile for 5% profit rake"


In [97]:
def has_place_v2(MarketStr):
    """If market string has a place - marks as a place bet. Otherwise win"""
    if "place" in MarketStr.lower():
        return "Place"
    return "Win"

def clean_horse(HorseStr):
    """If market string has a place - marks as a place bet. Otherwise win"""
    if "place" in HorseStr.lower():
        return HorseStr[:-8].strip()
    return HorseStr.strip()

betting_results["Market Type"]=betting_results["Horse"].apply(has_place_v2)
betting_results["Horse"]=betting_results["Horse"].apply(clean_horse)


temp_df=betting_results.loc[:,['Horse','Market Type','Victor Stake']].groupby(by=["Horse",'Market Type']).sum()
temp_df=temp_df.rename({'Victor Stake':"Total Stake"},axis=1)
betting_results=betting_results.join(temp_df,on=["Horse",'Market Type'])
betting_results["Pct Stake"]=betting_results["Victor Stake"]/betting_results["Total Stake"]




betting_results.sort_values("Horse").head()

,Horse,Odds,Victor Stake,Result,Notes,Market Type,Total Stake,Pct Stake
0,Bella Nipotina,6.0,133.0,Lost,Eligibile for 5% profit rake,Place,633.0,0.210111
47,Bella Nipotina,6.5,500.0,Lost,Victor Own,Place,633.0,0.789889
1,Collette,7.0,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Win,250.0,1.000000
2,Collette,2.5,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Place,250.0,1.000000
3,Counttheheadlights,41.0,34.5,Lost,NaN,Win,34.5,1.000000


In [98]:
results_df=(betting_results.merge(betfair_results,
                                    left_on=["Horse","Market Type"],
                                    right_on=["Selection","Market Type"],
                                    how="outer")
                           .drop("Selection",axis=1))
results_df.head(10)

,Horse,Odds,Victor Stake,Result,Notes,Market Type,Total Stake,Pct Stake,Bid type,Venue,Profit/Loss (AUD),Commission Paid,Avg. odds matched,Stake (AUD),Profit ex. Commission
0,Bella Nipotina,6.00,133.0,Lost,Eligibile for 5% profit rake,Place,633.0,0.210111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bella Nipotina,6.50,500.0,Lost,Victor Own,Place,633.0,0.789889,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Collette,7.00,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Win,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Collette,2.50,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Place,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Counttheheadlights,41.00,34.5,Lost,NaN,Win,34.5,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Ellsberg,6.48,300.0,Win,"Placed by Victor, Eligibile for 5% profit rake",Win,466.0,0.643777,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99
6,Ellsberg,6.50,166.0,Win,Eligibile for 5% profit rake,Win,466.0,0.356223,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99
7,Ellsberg,2.30,166.0,Win,Eligibile for 5% profit rake,Place,166.0,1.000000,Stake_lay,Randwick,-42.29,0.0,1.28,150.91,-42.29
8,I'm Thunder Struck,4.50,335.0,Lost,Eligibile for 5% profit rake,Win,335.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,I'm Thunder Struck,1.90,670.0,Lost,NaN,Place,670.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
def calc_liability_est(df):
    if df["Bid type"] != "Full_lay":
        return 0
    return df["Odds"]*df["Victor Stake"]/(1+((1-commission_dict[df["Venue"]])/(df["Avg. odds matched"]-1)))
    

results_df["Full Lay Liability"]=results_df.apply(calc_liability_est,axis=1)

temp_df=results_df.loc[:,['Horse','Market Type',"Full Lay Liability"]].groupby(by=["Horse",'Market Type']).sum()
temp_df=temp_df.rename({"Full Lay Liability":"Total Expected Liability"},axis=1)
results_df=results_df.join(temp_df,on=["Horse",'Market Type'])
results_df["Liability pctg"]=results_df.apply(lambda x: x["Full Lay Liability"]/x["Total Expected Liability"] if x["Full Lay Liability"] != 0 else 0,axis=1)
results_df.head(10)


,Horse,Odds,Victor Stake,Result,Notes,Market Type,Total Stake,Pct Stake,Bid type,Venue,Profit/Loss (AUD),Commission Paid,Avg. odds matched,Stake (AUD),Profit ex. Commission,Full Lay Liability,Total Expected Liability,Liability pctg
0,Bella Nipotina,6.00,133.0,Lost,Eligibile for 5% profit rake,Place,633.0,0.210111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,Bella Nipotina,6.50,500.0,Lost,Victor Own,Place,633.0,0.789889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,Collette,7.00,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Win,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,Collette,2.50,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Place,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,Counttheheadlights,41.00,34.5,Lost,NaN,Win,34.5,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
5,Ellsberg,6.48,300.0,Win,"Placed by Victor, Eligibile for 5% profit rake",Win,466.0,0.643777,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99,0.0,0.0,0.0
6,Ellsberg,6.50,166.0,Win,Eligibile for 5% profit rake,Win,466.0,0.356223,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99,0.0,0.0,0.0
7,Ellsberg,2.30,166.0,Win,Eligibile for 5% profit rake,Place,166.0,1.000000,Stake_lay,Randwick,-42.29,0.0,1.28,150.91,-42.29,0.0,0.0,0.0
8,I'm Thunder Struck,4.50,335.0,Lost,Eligibile for 5% profit rake,Win,335.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
9,I'm Thunder Struck,1.90,670.0,Lost,NaN,Place,670.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [100]:
def calc_betfair_result(df):
    if df["Bid type"]=="Stake_lay":
        return df["Pct Stake"]*df["Profit ex. Commission"]
    elif df["Bid type"]=="Full_lay":
        return df["Liability pctg"]*df["Profit ex. Commission"]
    return 0

results_df["Betfair Result"]=results_df.apply(calc_betfair_result,axis=1)
results_df.head(20)

,Horse,Odds,Victor Stake,Result,Notes,Market Type,Total Stake,Pct Stake,Bid type,Venue,Profit/Loss (AUD),Commission Paid,Avg. odds matched,Stake (AUD),Profit ex. Commission,Full Lay Liability,Total Expected Liability,Liability pctg,Betfair Result
0,Bella Nipotina,6.00,133.0,Lost,Eligibile for 5% profit rake,Place,633.0,0.210111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
1,Bella Nipotina,6.50,500.0,Lost,Victor Own,Place,633.0,0.789889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
2,Collette,7.00,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Win,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
3,Collette,2.50,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Place,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
4,Counttheheadlights,41.00,34.5,Lost,NaN,Win,34.5,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
5,Ellsberg,6.48,300.0,Win,"Placed by Victor, Eligibile for 5% profit rake",Win,466.0,0.643777,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99,0.0,0.0,0.0,-581.967811
6,Ellsberg,6.50,166.0,Win,Eligibile for 5% profit rake,Win,466.0,0.356223,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99,0.0,0.0,0.0,-322.022189
7,Ellsberg,2.30,166.0,Win,Eligibile for 5% profit rake,Place,166.0,1.000000,Stake_lay,Randwick,-42.29,0.0,1.28,150.91,-42.29,0.0,0.0,0.0,-42.290000
8,I'm Thunder Struck,4.50,335.0,Lost,Eligibile for 5% profit rake,Win,335.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000
9,I'm Thunder Struck,1.90,670.0,Lost,NaN,Place,670.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000


In [101]:
results_df["Result no arb"]=results_df.apply(lambda x: x["Victor Stake"]*(x["Odds"]-1) if x["Result"].lower() == "win" else -x["Victor Stake"],axis=1)
results_df.head(20)

,Horse,Odds,Victor Stake,Result,Notes,Market Type,Total Stake,Pct Stake,Bid type,Venue,Profit/Loss (AUD),Commission Paid,Avg. odds matched,Stake (AUD),Profit ex. Commission,Full Lay Liability,Total Expected Liability,Liability pctg,Betfair Result,Result no arb
0,Bella Nipotina,6.00,133.0,Lost,Eligibile for 5% profit rake,Place,633.0,0.210111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-133.0
1,Bella Nipotina,6.50,500.0,Lost,Victor Own,Place,633.0,0.789889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-500.0
2,Collette,7.00,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Win,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-250.0
3,Collette,2.50,250.0,Lost,"Placed by Victor, Eligibile for 5% profit rake",Place,250.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-250.0
4,Counttheheadlights,41.00,34.5,Lost,NaN,Win,34.5,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-34.5
5,Ellsberg,6.48,300.0,Win,"Placed by Victor, Eligibile for 5% profit rake",Win,466.0,0.643777,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99,0.0,0.0,0.0,-581.967811,1644.0
6,Ellsberg,6.50,166.0,Win,Eligibile for 5% profit rake,Win,466.0,0.356223,Stake_lay,Randwick,-903.99,0.0,2.76,513.64,-903.99,0.0,0.0,0.0,-322.022189,913.0
7,Ellsberg,2.30,166.0,Win,Eligibile for 5% profit rake,Place,166.0,1.000000,Stake_lay,Randwick,-42.29,0.0,1.28,150.91,-42.29,0.0,0.0,0.0,-42.290000,215.8
8,I'm Thunder Struck,4.50,335.0,Lost,Eligibile for 5% profit rake,Win,335.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-335.0
9,I'm Thunder Struck,1.90,670.0,Lost,NaN,Place,670.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,-670.0


In [102]:
def calc_kelvin_to_pay(df):
    if df["Notes"]=="Victor Own":
        return df["Betfair Result"]
    elif pd.isnull(df["Notes"]):
        return df["Betfair Result"]+df["Result no arb"]
    elif "victor" in df["Notes"].lower():
        return df["Betfair Result"]-df["Victor Stake"]
    else:
        return df["Betfair Result"]+df["Result no arb"]

results_df["Kelvin To Pay"]= results_df.apply(calc_kelvin_to_pay,axis=1)
results_df["Final Result"]=results_df["Betfair Result"]+results_df["Result no arb"]
results_df.to_excel("Final_calcs.xlsx",index=False)
